In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

c:\Users\189\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#загружаем обученную модель и токенизатор
model_path = 'TaskMaster-BERT2'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

In [4]:
#режим проверки (а не тренировки)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [28]:
#функция для обращения к модели
def get_prediction(text):
    #токенизируем инпут
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    #модель выдает результат
    with torch.no_grad():
        logits = model(**inputs).logits
    #берем самый большой по весу
    predicted_class_id = logits.argmax().item()
    return predicted_class_id

In [42]:
#функция с категорией "другое"
def get_prediction2(text, threshold=0.1):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        logits = model(**inputs).logits
    #сравниваем два самых больших результата    
    max_logit, max_idx = logits.max(dim=1)
    second_max_logit = logits[0].sort(descending=True)[0][1]
    
    #если разница меньше порога, ставим категорию другое
    if max_logit - second_max_logit < threshold:
        predicted_class_id = 3
        return predicted_class_id
    else:
        predicted_class_id = max_idx.item()
        return predicted_class_id

In [26]:
#функция для обозначения класса словом
def id_to_name(prediction):
    if prediction == 0:
        predicted_name = 'home'
    elif prediction == 1:
        predicted_name = 'work-study'
    elif prediction == 2:
        predicted_name = 'shopping'
    elif prediction == 3:
        predicted_name = 'other'
    return predicted_name

In [57]:
#проверяем работу
text = "организовать корпоратив"
prediction = get_prediction2(text)
tag = id_to_name(prediction)
print("Тег", tag)

Predicted class: 3 Predicted name: other
